# AUTOGRAD: AUTOMATIC DIFFERENTIATION / 자동 미분

> The autograd package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

> autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공합니다. 이는 실행-기반-정의(define-by-run) 프레임워크로, 이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라집니다.



# Tensor

Create a tensor and set requires_grad=True to track computation with it / tensor를 생성하고 requires_grad=True 를 설정하여 연산을 기록합니다.

In [0]:
import torch                                     # Always import "torch" package first

x = torch.ones(2, 2, requires_grad=True)         # requires_grad는 
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do a tensor operation: / tensor에 연산을 수행

In [0]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y was created as a result of an operation, so it has a grad_fn. / y 는 연산의 결과로 생성된 것이므로 grad_fn을 갖는다

In [0]:
print(y.grad_fn)

Do more operations on y / y 에 다른 연산을 수행



In [0]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


.requires_grad_( ... ) changes an existing Tensor’s requires_grad flag in-place. The input flag defaults to False if not given. / .requires_grad_( ... ) 는 기존 Tensor의 requires_grad 값을 바꿔치기 (in-place)하여 변경 입력값이 지정되지 않으면 기본값은 False 

In [0]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))

print(a.requires_grad)

a.requires_grad_(True)

print(a.requires_grad)

b = (a * a).sum()

print(b.grad_fn)

False
True


# Gradients / 변화도

이제 역전파(backprop)를 실행. out 은 하나의 스칼라 값만 갖고 있기 때문에, out.backward() 는 out.backward(torch.tensor(1.)) 과 동일

In [7]:
out.backward()

print(x.grad) # Print gradients d(out)/dx

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


* # Vector-Jacobian product / 벡터-야코비안 곱

---








In [8]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 1030.6512, -1299.9962,   805.2855], grad_fn=<MulBackward0>)


Now in this case y is no longer a scalar. torch.autograd could not compute the full Jacobian directly, but if we just want the vector-Jacobian product, simply pass the vector to backward as argument  


---


이 경우 y 는 더 이상 스칼라 값이 아닙니다. torch.autograd 는 전체 야코비안을 직접 계산할수는 없지만, 벡터-야코비안 곱은 간단히 backward 에 해당 벡터를 인자로 제공하여 얻을 수 있습니다.

In [9]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float) # tensor의 shape와 같음
y.backward(v)

print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


You can also stop autograd from tracking history on Tensors with .requires_grad=True either by wrapping the code block in with torch.no_grad():



---

또한 with torch.no_grad(): 로 코드 블럭을 감싸서 autograd가 .requires_grad=True 인 Tensor들의 연산 기록을 추적하는 것을 멈출 수 있습니다.



In [10]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


Or by using .detach() to get a new Tensor with the same content but that does not require gradients:



---

또는 .detach() 를 호출하여 내용물(content)은 같지만 require_grad가 다른 새로운 Tensor를 가져옵니다:

In [11]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)
